In [1]:
from xs_lexer import Lexer
from xs_parser import Parser
import copy
# from xs_node import Node_letter, Node_condition

In [2]:
class Node_letter(object):

    def __init__(self, token, name):
        self.token = token
        self.name = name
        self.state = 0
        self.neg = 0
        self.childs_pos = []
        self.childs_neg = []
        self.visited = 0
        self.or_dic = {}
        self.xor_dic = {}
        self.ind = 0

    def __str__(self):
        return "Node_letter({})".format(self.token)


class Node_condition(object):

    def __init__(self, left, cond, right):
        self.left = left
        self.token = self.cond = cond
        self.right = right
        self.neg = 0
        self.visited = 0
        self.or_dic = {}
        self.xor_dic = {}
        self.ind = 0

    def __str__(self):
        return "Node_condition({}".format(self.token)

In [3]:
class Interpreter(object):

    def __init__(self):
        self.graph_list = {}
        self.all_histories = []
        pass

    def interpret(self, node, graph):
        if node:
            if type(node).__name__ == "Node_condition":
                return self.apply_logical(node, node.left, node.right, graph)
            node.visited = 1 
            if type(node).__name__ == "Node_letter":
                implies_node = graph.find_letter_in_implies(node.name)
                if implies_node:
                    for child in implies_node.childs_pos:
                        if child.visited == 0:
                            if self.interpret(child, graph) == 1:
                                node.state = 1 
                    for child in implies_node.childs_neg:
                        if child.visited == 0:
                            if self.interpret(child, graph) == 1:
                                node.state = 0 
                node.visited = 0 
                return abs(node.state - node.neg)
            

    def apply_logical(self, node, left, right, graph):
        if node.token.type == "AND":
            return abs((self.interpret(left, graph) & self.interpret(right, graph)) - node.neg)
        if node.token.type == "OR":
            return abs((self.interpret(left, graph) | self.interpret(right, graph)) - node.neg)
        if node.token.type == "XOR":
            return abs((self.interpret(left, graph) ^ self.interpret(right, graph)) - node.neg)

In [4]:
rule1 = "A => B"
# rule2 = "F => Z"
# rule3 = "G | B => C"
# rule4 = "I => E ^ F"
rule5 = "!(!B + C)=> Z"
true_facts = "A"

queries = "BCZ"
try:
    graph = Graph()
    
    graph.add_new_AST(Parser(Lexer(rule1)).parse())
#     graph.add_new_AST(Parser(Lexer(rule2)).parse())
    graph.add_new_AST(Parser(Lexer(rule5)).parse())
    
    graph.learn_facts(true_facts)
    for node in graph.implies_list:
        graph.set_facts(node)
    if graph.incoherent == 0:
        graph.resolve_simple()
    else:
        graph.resolve_complex()

    for query in queries:
        graph.query(query)
    
except Exception as e:
    print(e)

name 'Graph' is not defined


In [5]:
for node in graph.implies_list:
    print(node)
print()
for node in graph.implies_list[0].childs_pos:
    print(hex(id(node)))

NameError: name 'graph' is not defined

In [45]:
graph = Graph()
    
graph.add_new_AST(Parser(Lexer(rule1)).parse())
# graph.add_new_AST(Parser(Lexer(rule2)).parse())
graph.add_new_AST(Parser(Lexer(rule5)).parse())
    
graph.learn_facts(true_facts)
for node in graph.implies_list:
    graph.set_facts(node)
if graph.incoherent == 0:
    graph.resolve_simple()
else:
    graph.resolve_complex()    
for query in queries:
    graph.query(query)

Node_letter(Token(LETTER, B)) Node_condition(Token(XOR, ^) 0 6
Node_letter(Token(LETTER, B)) {<xs_node.Node_letter object at 0x104310dd8>: <xs_node.Node_condition object at 0x1042b94a8>}
Node_letter(Token(LETTER, B)) {<xs_node.Node_letter object at 0x1042f6f98>: <xs_node.Node_condition object at 0x1042f6cc0>}
Node_letter(Token(LETTER, C)) Node_letter(Token(LETTER, D)) 6 0
Node_letter(Token(LETTER, C)) {<xs_node.Node_letter object at 0x1044446a0>: <xs_node.Node_letter object at 0x104444b38>}


TypeError: 'NoneType' object is not subscriptable

In [61]:
class Graph(object):

    def __init__(self):
        self.ast_list = []
        self.implies_list = []
        self.implies_name_list = []
        self.true_facts = ""
        self.incoherent = 0
        self.interpretor = Interpreter()
        self.full_history = {}
        self.list_copy = []
        self.list_copy_rule = []

    def error(self, s):
        raise Exception(s)

    def add_new_AST(self, root):
        self.update_implies_list(root.right)
        self.update_graph(root.left, root.right, 0)
        self.ast_list.append(root.right)
        
    def update_implies_list(self, node):
        if node:
            if type(node).__name__ == "Node_letter":
                if node.name not in self.implies_name_list:
                    self.implies_list.append(node)
                    self.implies_name_list.append(node.name)
            if type(node).__name__ == "Node_condition":
                self.update_implies_list(node.left)
                self.update_implies_list(node.right)

    def update_graph(self, left, node, neg):
        if node:
            if type(node).__name__ == "Node_letter":
                if abs(node.neg - neg) == 1:
                    (self.find_letter_in_implies(node.name)
                        .childs_neg.append(left))
                else:
                    (self.find_letter_in_implies(node.name)
                        .childs_pos.append(left))
            if type(node).__name__ == "Node_condition":
                if node.token.type == "OR" or node.token.type == "XOR":
                    self.update_ind_dics(left, node)
                    self.incoherent += 1
                self.update_graph(left, node.left, abs(node.neg - neg))
                self.update_graph(left, node.right, abs(node.neg - neg))

    def update_ind_dics(self, left, node):
        if node.token.type == "OR":
            if type(node.left).__name__ == "Node_letter":
                self.find_letter_in_implies(node.left.name).or_dic[left] = node.right
            else:
                node.left.or_dic[left] = node.right
            if type(node.right).__name__ == "Node_letter":
                self.find_letter_in_implies(node.right.name).or_dic[left] = node.left
            else:
                node.right.or_dic[left] = node.left
        if node.token.type == "XOR":
            if type(node.left).__name__ == "Node_letter":
                self.find_letter_in_implies(node.left.name).xor_dic[left] = node.right
            else:
                node.left.xor_dic[left] = node.right
            if type(node.right).__name__ == "Node_letter":
                self.find_letter_in_implies(node.right.name).xor_dic[left] = node.left
            else:
                node.right.xor_dic[left] = node.left
            
    def find_letter_in_implies(self, name):
        for node in self.implies_list:
            if node.name == name:
                return node
        return None

    def learn_facts(self, true_facts):
        self.true_facts = true_facts

    def set_facts(self, node):
        if node:
            if type(node).__name__ == "Node_letter":
                for child in (node.childs_pos + node.childs_neg):
                    self.set_facts(child)
                if node.token.value in self.true_facts:
                    node.state = 1 
            if type(node).__name__ == "Node_condition":
                self.set_facts(node.left)
                self.set_facts(node.right)
        
    def query(self, letter):
        if letter in self.full_history.keys():
            if self.full_history[letter] != None:
                print("{} is {}".format(letter, self.full_history[letter]))
                return self.full_history[letter]
        if letter in self.true_facts:
            print("{} is True".format(letter))
            return True
        else:
            print("{} is False".format(letter))
            return False

    def resolve_simple(self):
        for node in self.implies_list:
            self.full_history[node.name] = self.get_simple_final_state(node)
            if self.full_history[node.name] is None:
                if node.name in self.true_facts:
                    self.full_history[node.name] = True
                else:
                    self.full_history[node.name] = False
        return self.full_history
    
    def get_simple_final_state(self, node):
        history = []
        for child in node.childs_pos:
            res = self.interpretor.interpret(child, self)
            if res == 1:
                history.append(True)

        for child in node.childs_neg:
            res = self.interpretor.interpret(child, self)
            if res == 1:
                history.append(False)

        if len(set(history)) == 0:
            return
        elif len(set(history)) == 1:
            return history[0]
        else:
            self.error("Error contradiction found with letter {}"
                       .format(node.name))
   
    def merge_new_graph(self, new):
        if self.tmp_history == {}:
            self.tmp_history = new
            return
        for letter, state in new.items():
            if state != self.tmp_history[letter]:
                self.tmp_history[letter] = "IND"
                
    def resolve_complex(self):
        self.check_ind_rules(self, self, None, 0)
        self.tmp_history = {}
        for copy_graph in self.list_copy:
            hist = copy_graph.resolve_simple()
#             print(hist)
            self.merge_new_graph(hist)
        self.full_history = self.tmp_history

        
    """
    AAAAAAAAAA
    AAAAAAAAAA
    AAAAAAAAAAAA
    AAAAAAAAAAAA
    AAAAAAAAAAAAA
    IIICCCCIIIIICCCCIIIIICCCCII
    IIICCCCII
    """
    
    def handle_xor(self, origin_graph, current_graph, prev_rule, prev_case, node, rule, mirror):
        if prev_case == 1 or prev_case == 2 or prev_case == 6 or prev_case == 7:
            self.add_recursively_rule(node, prev_rule)
            self.add_recursively_rule(mirror, prev_rule)
        
        for index in range(0, len(current_graph.list_copy_rule)):
            if node in current_graph.list_copy_rule[index]:
                self.add_recursively_rule(node, current_graph.list_copy_rule[index][0])
                self.add_recursively_rule(mirror, current_graph.list_copy_rule[index][0])
                del current_graph.list_copy_rule[index]
                prev_case = 4
                break
                
        copy_graph_1 = copy.deepcopy(current_graph)
        copy_graph_2 = copy.deepcopy(current_graph)
        copy_rule = None
        case = 0
        if type(mirror).__name__ == "Node_condition" and type(node).__name__ == "Node_condition":
            case = 3
            if prev_case == 0 or prev_case == 3:
                copy_rule_1, copy_node_1 = self.set_special_xor(rule, node, current_graph, copy_graph_1, 1)
                copy_rule_2, copy_node_2 = self.set_special_xor(rule, node, current_graph, copy_graph_2, 2)                
                self.check_ind_rules(origin_graph, copy_graph_1, copy_rule_1, 3)
                self.check_ind_rules(origin_graph, copy_graph_2, copy_rule_2, 3)
            elif prev_case == 1 or prev_case == 2 or prev_case == 4 or prev_case == 5:
                copy_rule_1 = self.set_special_xor(rule, node, current_graph, copy_graph_1, 3)
                copy_rule_2 = self.set_special_xor(rule, node, current_graph, copy_graph_2, 4)
                self.check_ind_rules(origin_graph, copy_graph_1, copy_rule_1, 3)
                self.check_ind_rules(origin_graph, copy_graph_2, copy_rule_2, 3)                
                
        if type(mirror).__name__ == "Node_condition" and case != 3:
            copy_node = copy_graph_1.find_letter_in_implies(node.name)
            copy_rule = self.get_copy_rule(node, rule, copy_node, copy_graph_1)
            self.handle_neg(copy_rule, copy_node, 0)
            self.handle_neg(copy_rule, copy_node.xor_dic[copy_rule[0]], 1)
            if mirror.neg == 1:
                case = 6
            else:
                case = 1
            
        if type(node).__name__ == "Node_condition" and case != 3:
            copy_node = copy_graph_1.find_letter_in_implies(mirror.name)
            copy_rule = self.get_copy_rule(mirror, rule, copy_node, copy_graph_1)
            self.handle_neg(copy_rule, copy_node, 0)
            self.handle_neg(copy_rule, copy_node.xor_dic[copy_rule[0]], 1)
            if node.neg == 1:
                case = 7
            else:
                case = 2
        
#         print(node, mirror, prev_case, case)
    
        if prev_case == 0:
            if case == 0 or case == 1 or case == 6:
                self.set_rules_xor(rule, node, copy_graph_1, 1)
                self.set_rules_xor(rule, node, copy_graph_2, 2)
            elif case == 2 or case == 7:
                self.set_rules_xor(rule, mirror, copy_graph_1, 1)
                self.set_rules_xor(rule, mirror, copy_graph_2, 2)
        if prev_case == 1:
            if case == 0 or case == 1 or case == 6:
                self.set_rules_xor(rule, node, copy_graph_1, 3)
                self.set_rules_xor(rule, node, copy_graph_2, 4)
            elif case == 2 or case == 7:
                self.set_rules_xor(rule, mirror, copy_graph_1, 3)
                self.set_rules_xor(rule, mirror, copy_graph_2, 4)
        if prev_case == 2:
            if case == 0 or case == 1 or case == 6:
                self.set_rules_xor(rule, node, copy_graph_1, 3)
                self.set_rules_xor(rule, node, copy_graph_2, 4)
            elif case == 2 or case == 7:
                self.set_rules_xor(rule, mirror, copy_graph_1, 3)
                self.set_rules_xor(rule, mirror, copy_graph_2, 4)
        if prev_case == 3:
            if case == 0 or case == 1 or case == 6:
                self.set_rules_xor(rule, node, copy_graph_1, 1)
                self.set_rules_xor(rule, node, copy_graph_2, 2)
            elif case == 2 or case == 7:
                self.set_rules_xor(rule, mirror, copy_graph_1, 1)
                self.set_rules_xor(rule, mirror, copy_graph_2, 2)
        if prev_case == 4 or prev_case == 5:
            if case == 0 or case == 1:
                self.set_rules_xor(rule, node, copy_graph_1, 3)
                self.set_rules_xor(rule, node, copy_graph_2, 4)
            elif case == 2 or case == 7:
                self.set_rules_xor(rule, mirror, copy_graph_1, 3)
                self.set_rules_xor(rule, mirror, copy_graph_2, 4)
        if prev_case == 6:
            if case == 0 or case == 1 or case == 6:
                self.set_rules_xor(rule, node, copy_graph_1, 1)
                self.set_rules_xor(rule, node, copy_graph_2, 2)
            elif case == 2 or case == 7:
                self.set_rules_xor(rule, mirror, copy_graph_1, 1)
                self.set_rules_xor(rule, mirror, copy_graph_2, 2)
        if prev_case == 7:
            if case == 0 or case == 1 or case == 6:
                self.set_rules_xor(rule, node, copy_graph_1, 1)
                self.set_rules_xor(rule, node, copy_graph_2, 2)
            elif case == 2 or case == 7:
                self.set_rules_xor(rule, mirror, copy_graph_1, 1)
                self.set_rules_xor(rule, mirror, copy_graph_2, 2)
        
        if case == 0:
            self.check_ind_rules(origin_graph, copy_graph_1, copy_rule, case)
            self.check_ind_rules(origin_graph, copy_graph_2, copy_rule, case)
        if case == 1 or case == 5 or case == 6:
            self.check_ind_rules(origin_graph, copy_graph_1, copy_rule, case)
            self.check_ind_rules(origin_graph, copy_graph_2, None, 0)
        if case == 2 or case == 7:
            self.check_ind_rules(origin_graph, copy_graph_1, copy_rule, case)
            self.check_ind_rules(origin_graph, copy_graph_2, None, 0)
        
    """
    IIICCCCII
    IIICCCCII
    IIICCCCIIIIICCCCII
    IIICCCCII
    IIICCCCIIv
    v
    IIICCCCIIIIICCCCIIIIICCCCII
    IIICCCCII
    """
    
    def handle_or(self, origin_graph, current_graph, prev_rule, prev_case, node, rule, mirror):
        if prev_case == 1 or prev_case == 2:
            self.add_recursively_rule(node, prev_rule)
            self.add_recursively_rule(mirror, prev_rule)
        for index in range(0, len(current_graph.list_copy_rule)):
            if node in current_graph.list_copy_rule[index]:
                self.add_recursively_rule(node, current_graph.list_copy_rule[index][0])
                self.add_recursively_rule(mirror, current_graph.list_copy_rule[index][0])
                del current_graph.list_copy_rule[index]
                prev_case = 4
                break
        copy_graph_1 = copy.deepcopy(current_graph)
        copy_graph_2 = copy.deepcopy(current_graph)
        copy_graph_3 = copy.deepcopy(current_graph)
        copy_rule = None
        case = 0
        
        
        if type(mirror).__name__ == "Node_condition" and type(node).__name__ == "Node_condition":
            case = 3
            if prev_case == 0 or prev_case == 3:
                copy_rule_1 = self.set_special_or(rule, node, current_graph, copy_graph_1, 1)
                self.check_ind_rules(origin_graph, copy_graph_1, copy_rule_1, 3)
                copy_rule_2 = self.set_special_or(rule, node, current_graph, copy_graph_2, 2)
                self.check_ind_rules(origin_graph, copy_graph_2, copy_rule_2, 3)
                copy_rule_3 = self.set_special_or(rule, node, current_graph, copy_graph_3, 4)
                self.check_ind_rules(origin_graph, copy_graph_3, copy_rule_3, 3)
            
            elif prev_case == 1 or prev_case == 2 or prev_case == 4 or prev_case == 5:
                copy_rule_1 = self.set_special_or(rule, node, current_graph, copy_graph_1, 3)
                self.check_ind_rules(origin_graph, copy_graph_1, copy_rule_1, 3)

                
        if type(mirror).__name__ == "Node_condition" and case != 3:
            copy_node = copy_graph_1.find_letter_in_implies(node.name)
            copy_rule = self.get_copy_rule(node, rule, copy_node, copy_graph_1)
            self.handle_neg(copy_rule, copy_node, 0)
            self.handle_neg(copy_rule, copy_node.or_dic[copy_rule[0]], 1)
            case = 1 
        if type(node).__name__ == "Node_condition" and case != 3:
            copy_node = copy_graph_1.find_letter_in_implies(mirror.name)
            copy_rule = self.get_copy_rule(mirror, rule, copy_node, copy_graph_1)
            self.handle_neg(copy_rule, copy_node, 0)
            self.handle_neg(copy_rule, copy_node.or_dic[copy_rule[0]], 1)
            case = 2
        
        if prev_case == 0:
            if case == 1 or case == 0:
                self.set_rules_or(rule, node, copy_graph_1, 1)
                self.set_rules_or(rule, node, copy_graph_2, 2)
                self.set_rules_or(rule, node, copy_graph_3, 4)
            elif case == 2:
                self.set_rules_or(rule, mirror, copy_graph_1, 1)
                self.set_rules_or(rule, mirror, copy_graph_2, 2)
                self.set_rules_or(rule, mirror, copy_graph_3, 4)
        if prev_case == 1:
            if case == 0 or case == 1:
                self.set_rules_or(rule, node, copy_graph_1, 3)
            elif case == 2:
                self.set_rules_or(rule, mirror, copy_graph_1, 3)
        if prev_case == 2:
            if case == 0 or case == 1:
                self.set_rules_or(rule, node, copy_graph_1, 3)
            elif case == 2:
                self.set_rules_or(rule, mirror, copy_graph_1, 3)
        if prev_case == 3:
            if case == 0 or case == 1:
                self.set_rules_or(rule, node, copy_graph_1, 1)
                self.set_rules_or(rule, node, copy_graph_2, 2)
                self.set_rules_or(rule, node, copy_graph_3, 4)
            elif case == 2:
                self.set_rules_or(rule, mirror, copy_graph_1, 1)
                self.set_rules_or(rule, mirror, copy_graph_2, 2)
                self.set_rules_or(rule, mirror, copy_graph_3, 4)
        if prev_case == 4:
            if case == 0 or case == 1:
                self.set_rules_or(rule, node, copy_graph_1, 3)
            elif case == 2:
                self.set_rules_or(rule, mirror, copy_graph_1, 3)
        if prev_case == 5:
            if case == 0 or case == 1:
                self.set_rules_or(rule, node, copy_graph_1, 4)
            elif case == 2:
                self.set_rules_or(rule, mirror, copy_graph_1, 4)
        if case == 0:
            if prev_case == 0 or prev_case == 3:
                self.check_ind_rules(origin_graph, copy_graph_1, copy_rule, case)
                self.check_ind_rules(origin_graph, copy_graph_2, copy_rule, case)
                self.check_ind_rules(origin_graph, copy_graph_3, copy_rule, case)
            elif prev_case == 1 or prev_case == 2 or prev_case == 4 or prev_case == 5:
                self.check_ind_rules(origin_graph, copy_graph_1, copy_rule, case)
            
        if case == 1:
            if prev_case == 0 or prev_case == 3:
                self.check_ind_rules(origin_graph, copy_graph_1, copy_rule, case)
                self.check_ind_rules(origin_graph, copy_graph_2, None, 0)
                self.check_ind_rules(origin_graph, copy_graph_3, None, 0)
            elif prev_case == 1 or prev_case == 2 or prev_case == 4 or prev_case == 5:
                self.check_ind_rules(origin_graph, copy_graph_1, copy_rule, case)
        if case == 2:
            if prev_case == 0 or prev_case == 3:
                self.check_ind_rules(origin_graph, copy_graph_1, copy_rule, case)
                self.check_ind_rules(origin_graph, copy_graph_2, None, 0)
                self.check_ind_rules(origin_graph, copy_graph_3, None, 0)
            elif prev_case == 1 or prev_case == 2 or prev_case == 4 or prev_case == 5:
                self.check_ind_rules(origin_graph, copy_graph_1, copy_rule, case)
        
    def check_ind_rules(self, origin_graph, current_graph, prev_rule, prev_case):
        tot = 0
        
        for node in current_graph.ast_list:
            tot = self.crawl_ast(node, origin_graph, current_graph, prev_rule, prev_case)
            if tot != 0:
                return
        if tot == 0:
            origin_graph.list_copy.append(current_graph)
#             print(current_graph.resolve_simple())
    
    def crawl_ast(self, node, origin_graph, current_graph, prev_rule, prev_case):
        if node:
            for rule, mirror in node.xor_dic.items():
                self.handle_xor(origin_graph, current_graph, prev_rule, prev_case, node, rule, mirror)
                return 1
            for rule, mirror in node.or_dic.items():
                self.handle_or(origin_graph, current_graph, prev_rule, prev_case, node, rule, mirror)
                return 1
            
            if type(node).__name__ == "Node_condition":
                if node.neg == 1 and prev_case == 0:
                    prev_case = 5
                if self.crawl_ast(node.left, origin_graph, current_graph, prev_rule, prev_case) == 1:
                    return 1
                return self.crawl_ast(node.right, origin_graph, current_graph, prev_rule, prev_case)
            return 0
    
    def set_special_xor(self, rule, node, current_graph, copy_graph, step):
        copy_node = self.get_copy_cond(node, current_graph, copy_graph)
        copy_rule = self.get_copy_rule_cond(node, rule, copy_node, copy_graph)
        copy_mirror = copy_node.xor_dic[copy_rule[0]]
        self.handle_neg(copy_rule, copy_node, 0)
        self.handle_neg(copy_rule, copy_mirror, 1)

        if step == 1:
            self.del_rule_in_childs(copy_mirror, copy_rule)
            copy_graph.list_copy_rule.append([copy_rule, copy_mirror.left, copy_mirror.right])
        if step == 2:
            self.del_rule_in_childs(copy_node, copy_rule)
            copy_graph.list_copy_rule.append([copy_rule, copy_node.left, copy_node.right])
        if step == 3:
            self.del_rule_in_childs(copy_mirror, copy_rule)
            self.del_rule_in_childs(copy_node, copy_rule)
            copy_graph.list_copy_rule.append([copy_rule, copy_mirror.left, copy_mirror.right])
            copy_graph.list_copy_rule.append([copy_rule, copy_node.left, copy_node.right])
        if step == 4:
            pass
        copy_mirror.xor_dic.pop(copy_rule[0])
        copy_node.xor_dic.pop(copy_rule[0])
        return copy_rule, copy_node
    
    def set_special_or(self, rule, node, current_graph, copy_graph, step):
        copy_node = self.get_copy_cond(node, current_graph, copy_graph)
        copy_rule = self.get_copy_rule_cond(node, rule, copy_node, copy_graph)
        copy_mirror = copy_node.or_dic[copy_rule[0]]
        self.handle_neg(copy_rule, copy_node, 0)
        self.handle_neg(copy_rule, copy_mirror, 1)
        if step == 1:
            self.del_rule_in_childs(copy_mirror, copy_rule)
            copy_graph.list_copy_rule.append([copy_rule, copy_mirror.left, copy_mirror.right])
        if step == 2:
            self.del_rule_in_childs(copy_node, copy_rule)
            copy_graph.list_copy_rule.append([copy_rule, copy_node.left, copy_node.right])
        if step == 3:
            self.del_rule_in_childs(copy_mirror, copy_rule)
            self.del_rule_in_childs(copy_node, copy_rule)
            copy_graph.list_copy_rule.append([copy_rule, copy_mirror.left, copy_mirror.right])
            copy_graph.list_copy_rule.append([copy_rule, copy_node.left, copy_node.right])
        if step == 4:
            pass
        copy_mirror.or_dic.pop(copy_rule[0])
        copy_node.or_dic.pop(copy_rule[0])
        return copy_rule
    
    def set_rules_xor(self, rule, node, copy_graph, step):
        copy_node = copy_graph.find_letter_in_implies(node.name)
        copy_rule = self.get_copy_rule(node, rule, copy_node, copy_graph)
#         print(node, node.childs_pos, node.childs_neg, rule)
        copy_mirror = copy_node.xor_dic[copy_rule[0]]
        self.handle_neg(copy_rule, copy_node, 0)
        self.handle_neg(copy_rule, copy_mirror, 1)
        if step == 1:
            self.del_rule_in_childs(copy_mirror, copy_rule)
        if step == 2:
            self.del_rule_in_childs(copy_node, copy_rule)
        if step == 3:
            self.del_rule_in_childs(copy_mirror, copy_rule)
            self.del_rule_in_childs(copy_node, copy_rule)
        copy_mirror.xor_dic.pop(copy_rule[0])
        copy_node.xor_dic.pop(copy_rule[0])

    def set_rules_or(self, rule, node, copy_graph, step):
        copy_node = copy_graph.find_letter_in_implies(node.name)
        copy_rule = self.get_copy_rule(node, rule, copy_node, copy_graph)      
        copy_mirror = copy_node.or_dic[copy_rule[0]]
        self.handle_neg(copy_rule, copy_node, 0)
        self.handle_neg(copy_rule, copy_mirror, 1)

        if step == 1:
            self.del_rule_in_childs(copy_mirror, copy_rule)
        if step == 2:
            self.del_rule_in_childs(copy_node, copy_rule)
        if step == 3:
            self.del_rule_in_childs(copy_mirror, copy_rule)
            self.del_rule_in_childs(copy_node, copy_rule)
        if step == 4:
            pass
        
        copy_mirror.or_dic.pop(copy_rule[0])
        copy_node.or_dic.pop(copy_rule[0])
    
    def del_rule_in_childs(self, node, rule):
        if node in rule[1]:
            if type(node).__name__ == "Node_letter":
                for index in range(0, len(node.childs_neg)):
                    if node.childs_neg[index] == rule[0]:
                        del node.childs_neg[index]
                        if rule[0] not in node.childs_pos:
                            node.childs_pos.append(rule[0])
                        return
            else:
                self.delete_recursively_rule(node, rule)
        else:
            if type(node).__name__ == "Node_letter":
                for index in range(0, len(node.childs_pos)):
                    if node.childs_pos[index] == rule[0]:
                        del node.childs_pos[index]
                        return
            else:
                self.delete_recursively_rule(node, rule)
                
    def delete_recursively_rule(self, node, rule):
        if node:
            if type(node).__name__ == "Node_letter":
                self.del_rule_in_childs(node, rule)
            if type(node).__name__ == "Node_condition":
                self.delete_recursively_rule(node.left, rule)
                self.delete_recursively_rule(node.right, rule)
                
    def add_recursively_rule(self, node, rule):
        if node:
            if type(node).__name__ == "Node_letter":
 
                if node in rule[1]:
                    node.childs_neg.append(rule[0])
                    self.del_rule_in_childs(node, [rule[0], []])
                else:
                    node.childs_pos.append(rule[0])
            if type(node).__name__ == "Node_condition":
                self.add_recursively_rule(node.left, rule)
                self.add_recursively_rule(node.right, rule)
                
    def get_copy_rule(self, true_node, true_rule, copy_node, copy_graph):
        for index in range(0, len(true_node.childs_pos)):
            if true_node.childs_pos[index] == true_rule:
                return [copy_node.childs_pos[index], []]
        for index in range(0, len(true_node.childs_neg)):
            if true_node.childs_neg[index] == true_rule:
                return [copy_node.childs_neg[index], []]
        return 
    
    def get_copy_rule_cond(self, node, rule, copy_node, copy_graph):
        while type(node).__name__ == "Node_condition":
            node = node.left
            copy_node = copy_node.left
        return self.get_copy_rule(node, rule, copy_node, copy_graph)
        
    def get_copy_cond(self, node, current_graph, copy_graph):
        
        for i in range(0, len(current_graph.ast_list)):
            ret = self.find_cond_in_ast(node, current_graph.ast_list[i], copy_graph.ast_list[i])
            if ret:
                return ret
            
    def find_cond_in_ast(self, to_find, root, copy_root):
        if root:
            if to_find == root:
                return copy_root
            if type(root).__name__ == "Node_condition":
                ret = self.find_cond_in_ast(to_find, root.left, copy_root.left)
                if ret:
                    return ret
                return self.find_cond_in_ast(to_find, root.right, copy_root.right)
        return None
    
    def handle_neg(self, copy_rule, node, pos):
        if type(node).__name__ == "Node_letter":
            if copy_rule[0] in node.childs_neg :
                copy_rule[1].append(node)
        if type(node).__name__ == "Node_condition":
            self.handle_neg(copy_rule, node.left, pos)
            self.handle_neg(copy_rule, node.right, pos)

In [62]:
print(len(graph.list_copy))

4


In [63]:
# rule1 = "A => (((C ^ D) ^ I) ^ ((!E ^ F) ^ !(G ^ H)))"
# rule1 = "A => ((C ^ D) ^ ((!E | F) ^ (G | H)))"
rule1 = "A => !B ^ !(!C ^ !D)"
# rule2 = "A => D ^ E"
# rule3 = "H => C ^ D"
# rule4 = "I => E ^ F"
# rule5 = "(((C ^ D) ^ I) ^ ((!E ^ F) ^ !(G ^ H))) => Z"
rule5 = " !B ^ !(!C ^ !D) => Z"
true_facts = "A"

queries = "Z"
try:
    graph = Graph()
    
    graph.add_new_AST(Parser(Lexer(rule1)).parse())
#     graph.add_new_AST(Parser(Lexer(rule2)).parse())
#     graph.add_new_AST(Parser(Lexer(rule3)).parse())
#     graph.add_new_AST(Parser(Lexer(rule4)).parse())
    graph.add_new_AST(Parser(Lexer(rule5)).parse())
#     graph.add_new_AST(Parser(Lexer(rule6)).parse())
    
    graph.learn_facts(true_facts)
    for node in graph.implies_list:
        graph.set_facts(node)
    
    if graph.incoherent == 0:
        graph.resolve_simple()
    else:
        graph.resolve_complex()
    
    for query in queries:
        graph.query(query)
    
except Exception as e:
    print(e)

Z is True


In [10]:
rule1 = "A => B"
# rule2 = "F => Z"
# rule3 = "G | B => C"
# rule4 = "I => E ^ F"
rule5 = "!(B + C)=> Z"
true_facts = "A"

queries = "Z"
try:
    graph = Graph()
    
    graph.add_new_AST(Parser(Lexer(rule1)).parse())
#     graph.add_new_AST(Parser(Lexer(rule2)).parse())
#     graph.add_new_AST(Parser(Lexer(rule3)).parse())
#     graph.add_new_AST(Parser(Lexer(rule4)).parse())
    graph.add_new_AST(Parser(Lexer(rule5)).parse())
#     graph.add_new_AST(Parser(Lexer(rule6)).parse())
    
    graph.learn_facts(true_facts)
    for node in graph.implies_list:
        graph.set_facts(node)
    
    if graph.incoherent == 0:
        graph.resolve_simple()
    else:
        graph.resolve_complex()
    
    for query in queries:
        graph.query(query)
    
except Exception as e:
    print(e)

Z is True


In [11]:
for node in graph.implies_list:
    print(node)
print()
for node in graph.implies_list[0].childs_pos:
    if node in graph.implies_list[0].xor_dic:
        print("ici", node)
    else:
        print("pas ici", node)
print()
# print(graph.implies_list[2].childs_pos)
print(graph.implies_list[0].xor_dic.keys())

Node_letter(Token(LETTER, B))
Node_letter(Token(LETTER, Z))

pas ici Node_letter(Token(LETTER, A))

dict_keys([])


In [12]:
import copy

In [13]:
print(graph)
print(graph.implies_list)

t = copy.deepcopy(graph)
print(t)
print(t.implies_list)

[<xs_node.Node_letter object at 0x1044bbc88>, <xs_node.Node_letter object at 0x1044911d0>]
[<xs_node.Node_letter object at 0x1043105c0>, <xs_node.Node_letter object at 0x104493780>]


In [14]:
t = copy.copy(graph.implies_list[0])

rule = next(iter(graph.implies_list[0].xor_dic))
print(rule)
for x in graph.implies_list:
    print(x, hex(id(x)), x.xor_dic)
print(graph.implies_list[0].xor_dic[rule].left)

StopIteration: 

In [15]:
def prefix_run(node):
    if node: 
        if type(node).__name__ == "Node_letter":
            print(node, node.xor_dic)
        if type(node).__name__ == "Node_condition":
            prefix_run(node.left)
            print(node, node.xor_dic)
            prefix_run(node.right)

In [16]:
for node in graph.ast_list:
    prefix_run(node)
    print()

Node_letter(Token(LETTER, B)) {}

Node_letter(Token(LETTER, Z)) {}



In [17]:
rule1 = "A => B"
# rule2 = "F => Z"
# rule3 = "G | B => C"
# rule4 = "I => E ^ F"
rule5 = "!(B + C)=> Z"
true_facts = "A"

queries = "Z"

In [18]:
def test_final(test):
    rule1 = "A => " + test
    rule2 = test + " => Z"
    
    true_facts = "A"
    
    queries = "Z"
    
    try:
        graph = Graph()

        graph.add_new_AST(Parser(Lexer(rule1)).parse())
        graph.add_new_AST(Parser(Lexer(rule2)).parse())

        graph.learn_facts(true_facts)
        for node in graph.implies_list:
            graph.set_facts(node)

        if graph.incoherent == 0:
            graph.resolve_simple()
        else:
            graph.resolve_complex()

        for query in queries:
            graph.query(query)

    except Exception as e:
        print(e)

In [66]:
## SIMPLE PART 
# only and
test1 = "B"
test2 = "B + C"
test3 = "B + C + D"

#Adding parenthesis
test4 = "B + (C + D)"
test5 = "(B + D) + (C + E)"
test6 = "(B + (C + D)) + ((E + F) + G)"

#Adding negations
test7 = "!B"
test8 = "!B + !C"
test9 = "!B + !C + !D"
test10 = "!B + (C + D)"
test11 = "B + !(C + D)"
test12 = "!B + !(C + D)"
test13 = "!(B + D) + (C + E)"
test14 = "(B + D) + !(C + E)"
test15 = "!(B + D) + !(C + E)"
test16 = "!(!B + D) + !(C + E)"
test17 = "!(!B + !D) + !(C + E)"
test18 = "!(!B + !D) + !(!C + !E)"

### BONUS PART
## xor part
# simple_xor
test19 = "B ^ C"
test20 = "B ^ C ^ D"
test21 = "B ^ C ^ D ^ E"
test22 = "B ^ C ^ D ^ E ^ F"

# adding parenthesis
# from right
test23 = "(B ^ C)"
test24 = "B ^ (C ^ D)"
test25 = "B ^ (C ^ (D ^ E))"
test26 = "B ^ (C ^ (D ^ (E ^ F)))"
# from left
test27 = "(B ^ C) ^ D"
test28 = "((B ^ C) ^ D) ^ E"
test29 = "(((B ^ C) ^ D) ^ E) ^ F"
# from middle
test30 = "(B ^ C) ^ (D ^ E)"
test31 = "((B ^ C) ^ (D ^ E)) ^ ((F ^ G) ^ (H ^ I))"
# mixed
test32 = "(B ^ (C ^ D)) ^ E"
test33 = "((B ^ C) ^ D) ^ E"
test34 = "B ^ ((C ^ D) ^ E)"
test35 = "B ^ (C ^ (D ^ E))"
test36 = "(B ^ C) ^ ((D ^ E) ^ F)"
test37 = "(B ^ C) ^ (D ^ (E ^ F))"
test38 = "((B ^ C) ^ D) ^ (E ^ F)"
test39 = "(B ^ (C ^ D)) ^ (E ^ F)"
test40 = "(B ^ (C ^ D) ^ E) ^ (F ^ (G ^ H ) ^ I)"

# adding negations
# simple
test41 = "!B ^ C"
test42 = "B ^ !C"
test43 = "!B ^ !C"
#from right
test44 = "!B ^ (C ^ D)"
test45 = "B ^ !(C ^ D)"
test46 = "!B ^ !(C ^ D)"
test47 = "B ^ (!C ^ D)"
test48 = "B ^ (C ^ !D)"
test49 = "B ^ (!C ^ !D)"
test50 = "!B ^ (!C ^ !D)"
test51 = "!B ^ !(!C ^ !D)"
#from middle
test52 = "!(B ^ C) ^ (D ^ E)"
test53 = "(B ^ C) ^ !(D ^ E)"
test54 = "(!B ^ !C) ^ (D ^ E)"
test55 = "(B ^ C) ^ (!D ^ !E)"
test56 = "!(B ^ C) ^ !(D ^ E)"
test57 = "(!B ^ !C) ^ (!D ^ !E)"
test58 = "!(B ^ C) ^ (!D ^ !E)"
test59 = "(!B ^ !C) ^ !(D ^ E)"
#from left
test60 = "(B ^ C) ^ !D"
test61 = "!(B ^ C) ^ D"
test62 = "!(B ^ C) ^ !D"
test63 = "(!B ^ C) ^ D"
test64 = "(B ^ !C) ^ D"
test65 = "(!B ^ !C) ^ D"
test66 = "(!B ^ !C) ^ !D"
test67 = "!(!B ^ !C) ^ !D"

## or part
# simple_or
test69 = "B | C"
test80 = "B | C | D"
test81 = "B | C | D | E"
test82 = "B | C | D | E | F"

# adding parenthesis
# from right
test83 = "(B | C)"
test84 = "B | (C | D)"
test85 = "B | (C | (D | E))"
test86 = "B | (C | (D | (E | F)))"
# from left
test87 = "(B | C) | D"
test88 = "((B | C) | D) | E"
test89 = "(((B | C) | D) | E) | F"
# from middle
test90 = "(B | C) | (D | E)"
test91 = "((B | C) | (D | E)) | ((F | G) | (H | I))"
# mixed
test92 = "(B | (C | D)) | E"
test93 = "((B | C) | D) | E"
test94 = "B | ((C | D) | E)"
test95 = "B | (C | (D | E))"
test96 = "(B | C) | ((D | E) | F)"
test97 = "(B | C) | (D | (E | F))"
test98 = "((B | C) | D) | (E | F)"
test99 = "(B | (C | D)) | (E | F)"
test100 = "(B | (C | D) | E) | (F | (G | H ) | I)"

# adding negations
# simple
test101 = "!B | C"
test102 = "B | !C"
test103 = "!B | !C"
#from right
test104 = "!B | (C | D)"
test105 = "B | !(C | D)"
test106 = "!B | !(C | D)"
test107 = "B | (!C | D)"
test108 = "B | (C | !D)"
test109 = "B | (!C | !D)"
test110 = "!B | (!C | !D)"
test111 = "!B | !(!C | !D)"
#from middle
test112 = "!(B | C) | (D | E)"
test113 = "(B | C) | !(D | E)"
test114 = "(!B | !C) | (D | E)"
test115 = "(B | C) | (!D | !E)"
test116 = "!(B | C) | !(D | E)"
test117 = "(!B | !C) | (!D | !E)"
test118 = "!(B | C) | (!D | !E)"
test119 = "(!B | !C) | !(D | E)"
#from left
test120 = "(B | C) | !D"
test121 = "!(B | C) | D"
test122 = "!(B | C) | !D"
test123 = "(!B | C) | D"
test124 = "(B | !C) | D"
test125 = "(!B | !C) | D"
test126 = "(!B | !C) | !D"
test127 = "!(!B | !C) | !D"

In [71]:
print("test 1 : ", end='')
test_final(test1)
print("test 2 : ", end='')
test_final(test2)
print("test 3 : ", end='')
test_final(test3)
print("test 4 : ", end='')
test_final(test4)
print("test 5 : ", end='')
test_final(test5)
print("test 6 : ", end='')
test_final(test6)
print("test 7 : ", end='')
test_final(test7)
print("test 8 : ", end='')
test_final(test8)
print("test 9 : ", end='')
test_final(test9)
print("test 10 : ", end='')
test_final(test10)
print("test 11 : ", end='')
test_final(test11)
print("test 12 : ", end='')
test_final(test12)
print("test 13 : ", end='')
test_final(test13)
print("test 14 : ", end='')
test_final(test14)
print("test 15 : ", end='')
test_final(test15)
print("test 16 : ", end='')
test_final(test16)
print("test 17 : ", end='')
test_final(test17)
print("test 18 : ", end='')
test_final(test18)
print("########### XOR PART #################")

print("test 19 : ", end='')
test_final(test19)
print("test 20 : ", end='')
test_final(test20)
print("test 21 : ", end='')
test_final(test21)
print("test 22 : ", end='')
test_final(test22)
print("test 23 : ", end='')
test_final(test23)
print("test 24 : ", end='')
test_final(test24)
print("test 25 : ", end='')
test_final(test25)
print("test 26 : ", end='')
test_final(test26)
print("test 27 : ", end='')
test_final(test27)
print("test 28 : ", end='')
test_final(test28)
print("test 29 : ", end='')
test_final(test29)
print("test 30 : ", end='')
test_final(test30)
print("test 31 : ", end='')
test_final(test31)
print("test 32 : ", end='')
test_final(test32)
print("test 33 : ", end='')
test_final(test33)
print("test 34 : ", end='')
test_final(test34)
print("test 35 : ", end='')
test_final(test35)
print("test 36 : ", end='')
test_final(test36)
print("test 37 : ", end='')
test_final(test37)
print("test 38 : ", end='')
test_final(test38)
print("test 39 : ", end='')
test_final(test39)
print("test 40 : ", end='')
test_final(test40)
print("test 41 : ", end='')
test_final(test41)
print("test 42 : ", end='')
test_final(test42)
print("test 43 : ", end='')
test_final(test43)
print("test 44 : ", end='')
test_final(test44)
print("test 45 : ", end='')
test_final(test45)
print("test 46 : ", end='')
test_final(test46)
print("test 47 : ", end='')
test_final(test47)
print("test 48 : ", end='')
test_final(test48)
print("test 49 : ", end='')
test_final(test49)
print("test 50 : ", end='')
test_final(test50)
print("test 51 : ", end='')
test_final(test51)
# print("test 52 : ", end='')
# test_final(test52)
# print("test 53 : ", end='')
# test_final(test53)
# print("test 54 : ", end='')
# test_final(test54)
# print("test 55 : ", end='')
# test_final(test55)
# print("test 56 : ", end='')
# test_final(test56)
# print("test 57 : ", end='')
# test_final(test57)
# print("test 58 : ", end='')
# test_final(test58)
# print("test 59 : ", end='')
# test_final(test59)
print("test 60 : ", end='')
test_final(test60)
print("test 61 : ", end='')
test_final(test61)
print("test 62 : ", end='')
test_final(test62)
print("test 63 : ", end='')
test_final(test63)
print("test 64 : ", end='')
test_final(test64)
print("test 65 : ", end='')
test_final(test65)
print("test 66 : ", end='')
test_final(test66)
print("test 67 : ", end='')
test_final(test67)
print("########### OR PART #################")

print("test 69 : ", end='')
test_final(test69)
print("test 80 : ", end='')
test_final(test80)
print("test 81 : ", end='')
test_final(test81)
print("test 82 : ", end='')
test_final(test82)
print("test 83 : ", end='')
test_final(test83)
print("test 84 : ", end='')
test_final(test84)
print("test 85 : ", end='')
test_final(test85)
print("test 86 : ", end='')
test_final(test86)
print("test 87 : ", end='')
test_final(test87)
print("test 88 : ", end='')
test_final(test88)
print("test 89 : ", end='')
test_final(test89)
print("test 90 : ", end='')
test_final(test90)
print("test 91 : ", end='')
test_final(test91)
print("test 92 : ", end='')
test_final(test92)
print("test 93 : ", end='')
test_final(test93)
print("test 94 : ", end='')
test_final(test94)
print("test 95 : ", end='')
test_final(test95)
print("test 96 : ", end='')
test_final(test96)
print("test 97 : ", end='')
test_final(test97)
print("test 98 : ", end='')
test_final(test98)
print("test 99 : ", end='')
test_final(test99)
print("test 100 : ", end='')
test_final(test100)
print("test 101 : ", end='')
test_final(test101)
print("test 102 : ", end='')
test_final(test102)
print("test 103 : ", end='')
test_final(test103)
print("test 104 : ", end='')
test_final(test104)
print("test 105 : ", end='')
test_final(test105)
print("test 106 : ", end='')
test_final(test106)
print("test 107 : ", end='')
test_final(test107)
print("test 108 : ", end='')
test_final(test108)
print("test 109 : ", end='')
test_final(test109)
print("test 110 : ", end='')
test_final(test110)
print("test 111 : ", end='')
test_final(test111)
# print("test 112 : ", end='')
# test_final(test112)
# print("test 113 : ", end='')
# test_final(test113)
# print("test 114 : ", end='')
# test_final(test114)
# print("test 115 : ", end='')
# test_final(test115)
# print("test 116 : ", end='')
# test_final(test116)
# print("test 117 : ", end='')
# test_final(test117)
# print("test 118 : ", end='')
# test_final(test118)
# print("test 119 : ", end='')
# test_final(test119)
print("test 120 : ", end='')
test_final(test120)
print("test 121 : ", end='')
test_final(test121)
print("test 122 : ", end='')
test_final(test122)
print("test 123 : ", end='')
test_final(test123)
print("test 124 : ", end='')
test_final(test124)
print("test 125 : ", end='')
test_final(test125)
print("test 126 : ", end='')
test_final(test126)
print("test 127 : ", end='')
test_final(test127)


test 1 : Z is True
test 2 : Z is True
test 3 : Z is True
test 4 : Z is True
test 5 : Z is True
test 6 : Z is True
test 7 : Z is True
test 8 : Z is True
test 9 : Z is True
test 10 : Z is True
test 11 : Z is True
test 12 : Z is True
test 13 : Z is True
test 14 : Z is True
test 15 : Z is True
test 16 : Z is True
test 17 : Z is True
test 18 : Z is True
########### XOR PART #################
test 19 : Z is True
test 20 : Z is True
test 21 : Z is True
test 22 : Z is True
test 23 : Z is True
test 24 : Z is True
test 25 : Z is True
test 26 : Z is True
test 27 : Z is True
test 28 : Z is True
test 29 : Z is True
test 30 : Z is True
test 31 : Z is True
test 32 : Z is True
test 33 : Z is True
test 34 : Z is True
test 35 : Z is True
test 36 : Z is True
test 37 : Z is True
test 38 : Z is True
test 39 : Z is True
test 40 : Z is True
test 41 : Z is True
test 42 : Z is True
test 43 : Z is True
test 44 : Z is True
test 45 : Z is True
test 46 : Z is True
test 47 : Z is True
test 48 : Z is True
test 49 : 